In [1]:
import numpy
import pandas as pd
import json

from geopy.geocoders import Nominatim

import matplotlib as cm
import matplotlib as colors

from sklearn.cluster import KMeans

import folium

print ("Library Loaded")

Library Loaded


In [2]:
trnto_df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='html5lib')
trnto_df[0].to_csv('trnto.csv')
trnto_data=pd.read_csv('trnto.csv')

print (trnto_data.shape)
trnto_data.head()

(180, 4)


,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
NullBorough=trnto_data[trnto_data['Borough'] == 'Not assigned'].index
trnto_data.drop(NullBorough, axis=0, inplace=True)
print(trnto_data.shape)
trnto_data.head()



(103, 4)


,Unnamed: 0,Postal Code,Borough,Neighbourhood
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = trnto_data['Postal Code'].tolist()
latitudes = []
longitudes = []
print("pgeocode downloaded")


pgeocode downloaded


In [5]:
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        #print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
        
print (len(postal_codes),len(latitudes),len(longitudes))

trnto_data['Neighbourhood Latitude']=latitudes
trnto_data['Neighbourhood Longitude']=longitudes
trnto_data.head(103)

103 103 103


,Unnamed: 0,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude
2,2,M3A,North York,Parkwoods,43.7545,-79.3300
3,3,M4A,North York,Victoria Village,43.7276,-79.3148
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...,...
160,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
165,165,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
168,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
169,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


In [6]:
dt = trnto_data[trnto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print (dt.shape)
dt.head()

(19, 6)


,Unnamed: 0,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude
0,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,22,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,31,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754


In [8]:
CLIENT_ID = '4MXLLZOVKLRCHM1PXPCXJW5PMUYGDJN0ODFFJATRPPKOSUPR' # your Foursquare ID
CLIENT_SECRET = '0NMMWU2MKDJ0BZQVRZ4CICVAA3VLN5FY4XUKNLG1DUMDKFZF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
dt_venues = getNearbyVenues(names = dt['Neighbourhood'], latitudes=dt['Neighbourhood Latitude'], longitudes=dt['Neighbourhood Longitude'])
print(dt_venues.shape) 
dt_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
(1166, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [10]:
dt_venues.groupby('Neighbourhood').count()
print('There are {} unique categories.'.format(len(dt_venues['Venue Category'].unique())))

There are 178 unique categories.


In [11]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="trnto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
import folium

map_trnto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map for venue in Downtown toronto
for lat, lng, label in zip(dt_venues['Venue Latitude'], dt_venues['Venue Longitude'], dt_venues['Venue Category']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trnto)  
  
map_trnto

In [13]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighbourhood'] = dt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
manhattan_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"Regent Park, Harbourfront"


In [14]:
dt_onehot.shape

(1166, 179)

In [15]:
dt_grouped = dt_onehot.groupby('Neighbourhood').mean().reset_index()
dt_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,...,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.010638,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,...,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.017241
2,Central Bay Street,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.016949,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026667
5,"Commerce Court, Victoria Hotel",0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.019231,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.057692,0.000000,0.038462,0.000000,0.019231,0.000000,0.000000,0.000000


In [16]:
dt_grouped.shape

(19, 179)

In [18]:
topfive_venues = 5 #top 5 venues in Downtown Toronto

for hood in dt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(topfive_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1              Bakery  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4               Hotel  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                 Bar  0.05
3                Café  0.05
4           Speakeasy  0.03


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.20
1         Italian Restaurant  0.03
2                       Café  0.03
3  Middle Eastern Restaurant  0.03
4             Sandwich Place  0.03


----Christie----
           venue  freq
0           Café  0.25
1  Grocery Store  0.25
2     Baby Store  0.08
3    Coffee Shop  0.08
4    Candy Store  0.08


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.07
1     Sushi Restaurant  0.05
2          Co

In [19]:
def return_most_common_venues(row, topfive_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:topfive_venues]

In [20]:
import numpy as np

newtopten_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top 10 venues
columns = ['Neighbourhood']
for ind in np.arange(newtopten_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = dt_grouped['Neighbourhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], newtopten_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Café,Seafood Restaurant,Hotel,Restaurant,Beer Bar,Cocktail Bar,Japanese Restaurant,Art Gallery
1,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Bar,Café,Speakeasy,Park,French Restaurant,Grocery Store,Gym / Fitness Center,Bank
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Sandwich Place,Restaurant,Bubble Tea Shop,Plaza,Shopping Mall,Shoe Store
3,Christie,Café,Grocery Store,Baby Store,Coffee Shop,Candy Store,Park,Playground,Athletics & Sports,Performing Arts Venue,Museum
4,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Gay Bar,Mediterranean Restaurant,Men's Store,Hotel,Grocery Store,Yoga Studio


In [21]:
from sklearn.cluster import KMeans #Run k-means to cluster the neighborhood into 5 clusters.

kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 4, 3, 0, 0, 0, 0, 2, 0, 0, 4, 0, 1, 0, 0, 4, 0, 0])

In [22]:
import pandas 
# add clustering labels

neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

dt_merged = dt

# merge dt_grouped with dt to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dt_merged.head() # check the last columns!

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,4,Coffee Shop,Breakfast Spot,Dance Studio,Spa,Food Truck,Restaurant,Event Space,Greek Restaurant,Beer Store,Thai Restaurant
1,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Italian Restaurant,Sushi Restaurant,Diner,College Cafeteria,Mexican Restaurant,Burrito Place,Café,Chinese Restaurant,Coffee Shop,Creperie
2,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Italian Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Café,Lingerie Store,Sandwich Place
3,22,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Café,Coffee Shop,Seafood Restaurant,American Restaurant,Italian Restaurant,Cocktail Bar,Gastropub,Park,Sporting Goods Shop,Department Store
4,31,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,0,Coffee Shop,Bakery,Café,Seafood Restaurant,Hotel,Restaurant,Beer Bar,Cocktail Bar,Japanese Restaurant,Art Gallery


In [33]:
import folium
from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors 

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [mcolors.to_hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Neighbourhood Latitude'], dt_merged['Neighbourhood Longitude'], dt_merged['Neighbourhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
#What has the most clusters
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]


,Postal Code,Neighbourhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M7A,-79.3889,0,Italian Restaurant,Sushi Restaurant,Diner,College Cafeteria,Mexican Restaurant,Burrito Place,Café,Chinese Restaurant,Coffee Shop,Creperie
2,M5B,-79.3783,0,Coffee Shop,Clothing Store,Italian Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Café,Lingerie Store,Sandwich Place
3,M5C,-79.3756,0,Café,Coffee Shop,Seafood Restaurant,American Restaurant,Italian Restaurant,Cocktail Bar,Gastropub,Park,Sporting Goods Shop,Department Store
4,M5E,-79.3754,0,Coffee Shop,Bakery,Café,Seafood Restaurant,Hotel,Restaurant,Beer Bar,Cocktail Bar,Japanese Restaurant,Art Gallery
7,M5H,-79.3833,0,Coffee Shop,Café,Restaurant,Hotel,Gym,Thai Restaurant,Sushi Restaurant,Steakhouse,American Restaurant,Salad Place
9,M5K,-79.3823,0,Coffee Shop,Hotel,Café,American Restaurant,Seafood Restaurant,Restaurant,Japanese Restaurant,Salad Place,Sushi Restaurant,Sporting Goods Shop
10,M5L,-79.3823,0,Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Deli / Bodega,Salad Place,Steakhouse,Seafood Restaurant
11,M5S,-79.3987,0,Café,Japanese Restaurant,Bakery,Bookstore,Theater,French Restaurant,Noodle House,Pub,Restaurant,Dessert Shop
12,M5T,-79.3978,0,Café,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Caribbean Restaurant,Grocery Store,Farmers Market,Vietnamese Restaurant,Bar
13,M5V,-79.3995,0,Italian Restaurant,Coffee Shop,Bar,Café,Speakeasy,Park,French Restaurant,Grocery Store,Gym / Fitness Center,Bank


In [35]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Postal Code,Neighbourhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M4W,-79.373,1,Playground,Candy Store,Park,Grocery Store,Music Venue,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


In [36]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Postal Code,Neighbourhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M5J,-79.3936,2,Music Venue,Café,Harbor / Marina,Park,American Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


In [37]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Postal Code,Neighbourhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M6G,-79.4205,3,Café,Grocery Store,Baby Store,Coffee Shop,Candy Store,Park,Playground,Athletics & Sports,Performing Arts Venue,Museum


In [38]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Postal Code,Neighbourhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.3626,4,Coffee Shop,Breakfast Spot,Dance Studio,Spa,Food Truck,Restaurant,Event Space,Greek Restaurant,Beer Store,Thai Restaurant
5,M5G,-79.3860,4,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Sandwich Place,Restaurant,Bubble Tea Shop,Plaza,Shopping Mall,Shoe Store
15,M5W,-79.3787,4,Coffee Shop,Restaurant,Gym,Hotel,Café,Deli / Bodega,Italian Restaurant,Sporting Goods Shop,Japanese Restaurant,Seafood Restaurant


In [40]:
print("Cluster 0 is the most common cluster and has many Coffee Shops followed by Restaurants.\n")




Cluster 0 is the most common cluster and has many Coffee Shops followed by Restaurants.

